In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from IPython.display import display

from __future__ import print_function
import os
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
import math

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def setup_data(dir, num_circuits, file_prefix, dir_with_gates, separate_bits, norm = 'max_norm'):
  onlyfiles = next(os.walk(dir))[2] #dir is your directory path as string

  # Currently, we only count files with frequencies
  if dir_with_gates:
    num_files = int(math.ceil((len(onlyfiles) - 1)/3))
  else:
    num_files = len(onlyfiles)

  for i in range(num_files): # Only FC files matter and there is an extra IDs file
    # Give unique indexes to each circuit in each file
    idx = i*num_circuits
    d = np.load(dir + "/" + file_prefix + str(i) + ".npy", allow_pickle=True)
    circuits = np.concatenate(d[:,0]).reshape(d.shape[0],2) + np.array([idx,idx+ int(num_circuits/2)]) 

    # Separate out cicruits and freq
    d = d[:,1:]

    # Stack
    if i == 0:
      det_c = np.copy(d)
      circ = np.copy(circuits)
    else:
      det_c = np.vstack((det_c, d))
      circ = np.vstack((circ, circuits))

  # Separate out frequencies by setting on each bit (instead of settings on pairs)
  if separate_bits == "True":
    det_c_copy = np.empty(det_c.shape)

    # 0 on qubit_0
    det_c_copy[:,0] = det_c[:,0] + det_c[:,2]

    # 1 on qubit_0
    det_c_copy[:,1] = det_c[:,1] + det_c[:,3]

    # 0 on qubit_1
    det_c_copy[:,2] = det_c[:,0] + det_c[:,1]

    # 1 on qubit_1
    det_c_copy[:,3] = det_c[:,2] + det_c[:,3]

    det_c = det_c_copy



  # Normalize frequencies
  if norm == "max_norm":
    det_c = det_c/np.max(det_c)
  elif norm == "col_max_norm":
    det_c = det_c/det_c.max(axis = 0)
  elif norm == "row_norm":
    det_c = det_c/np.sum(det_c[0,:])

  # Used only for testing
  elif norm == "no_norm":
    det_c = det_c

  return circ, det_c

def create_labels(label, size):
  return np.full((size,), label)

def create_dataset(features, label):
  y = create_labels(label, features.shape[0])
  return features, y

def combine_datasets(x1, y1, x2, y2):
  return np.vstack((x1,x2)), np.concatenate((y1,y2))

def binary_comparison_data_pipeline(freq_1, freq_2, test_frac = 0.2):
  x1, y1 = create_dataset(freq_1, 1)
  x2, y2 = create_dataset(freq_2, 0)
  x, y = combine_datasets(x1,y1, x2, y2)
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

def svm_expt(freq_1, freq_2, test_size = 0.2):
  X_train, X_test, y_train, y_test = binary_comparison_data_pipeline(freq_1, freq_2, test_frac = test_size)

  clf = svm.SVC()
  clf.fit(X_train, y_train)

  test_acc = (clf.predict(X_test) == y_test).mean()

  return test_acc

# Apply binary expt to all possible combinations of data_list and pretty print
def create_expt_matrix(dataset_list, data_list, binary_expt):
  num_datasets = len(data_list)

  # get names of datasets
  dataset_names = [d['dir_name'] for d in dataset_list]

  expt_matrix = np.zeros((num_datasets, num_datasets))
  for i in range(num_datasets):
    for j in range(num_datasets):
      if (j >= i) :
        expt_matrix[i][j] = binary_expt(data_list[i][1], data_list[j][1])

  # pretty printing
  df = pd.DataFrame(expt_matrix, index=dataset_names, columns=dataset_names)
  display(df)

  return expt_matrix

# Setup a list of data to be experimented on

# dir_prefix - path to directory where all data is stored
# f_prefix - prefix of file names with frequency data
# dir_with_gates - True if data directory has gates included
# sep_bits - if True, setup data such that frequency is calculated on each result bit instead of result pairs
# normalize - Normalization scheme used
def setup_datalist(datasets, dir_prefix, f_prefix, sep_bits, normalize):
  data_list = []
  data_list.append(setup_data(dir_prefix + datasets[0]['dir_name'],  num_circuits = datasets[0]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[0]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[1]['dir_name'],  num_circuits = datasets[1]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[1]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[2]['dir_name'],  num_circuits = datasets[2]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[2]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[3]['dir_name'],  num_circuits = datasets[3]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[3]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[4]['dir_name'],  num_circuits = datasets[4]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[4]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[5]['dir_name'],  num_circuits = datasets[5]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[5]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[6]['dir_name'],  num_circuits = datasets[6]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[6]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[7]['dir_name'],  num_circuits = datasets[7]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[7]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))
  data_list.append(setup_data(dir_prefix + datasets[8]['dir_name'],  num_circuits = datasets[8]['num_circuits'], file_prefix = f_prefix, dir_with_gates = datasets[8]['dir_with_gates'], separate_bits = sep_bits, norm = normalize))

  return data_list

In [49]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = "row_norm")

In [50]:
ideal_freq.sum(axis = 1)

array([1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0], dtype=object)

In [46]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", separate_bits = "True", norm = "row_norm")

In [48]:
ideal_freq.sum(axis = 1)

array([1., 1., 1., ..., 1., 1., 1.])

In [53]:
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")

100


KeyboardInterrupt: ignored

## Normalization Schemes


1.   max_norm - takes the maximum over the entire frequency matrix (across all circuit settings and all result bit settings)
2.   col_max_norm - takes the columnwise maximum (i.e max across all circuit settings for a particular result setting)
3. row_norm - uses the row-wise sum (i.e total number of times a circuit setting was simulated)



## Metadata of Datasets used

In [4]:
datasets = [
            {
              'dir_name':"IdealGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"Operation Crosstalk 1 (a)",
              'num_circuits':20,
              'dir_with_gates':False
            },
            {
              'dir_name':"Operation Crosstalk 2 (b)",
              'num_circuits':20,
              'dir_with_gates':False
            },
            {
              'dir_name':"Detection Crosstalk (c)",
              'num_circuits':40,
              'dir_with_gates':False
            },
            {
              'dir_name':"XT1WithGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"XT2WithGates",
              'num_circuits':20,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetX400",
              'num_circuits':40,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetXWithGates",
              'num_circuits':40,
              'dir_with_gates':True
            },
            {
              'dir_name':"DetX100",
              'num_circuits':20,
              'dir_with_gates':True
            }
]

## Expts with bit frequencies not separated

In [ ]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates")
a_circuits, a_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False)
b_circuits, b_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False)
c_circuits, c_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False)
detx400_circuits, detx400_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40)
detxWithGates_circuits, detxWithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40)
detx100_circuits, detx100_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100")
xt1WithGates_circuits, xt1WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates")

In [5]:
dir_prefix = "gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/"
file_prefix = "FCTexts_"


### max_norm

In [14]:
separate_bits = False
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [17]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.99325,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.504750
Operation Crosstalk 1 (a),0.000,0.47950,0.99050,1.000000,1.00000,0.99075,0.992460,1.000000,0.988250
Operation Crosstalk 2 (b),0.000,0.00000,0.48550,1.000000,0.99850,0.50075,0.795437,1.000000,0.500250
Detection Crosstalk (c),0.000,0.00000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.00000,0.00000,0.000000,0.48800,0.99875,0.999702,1.000000,0.999000
XT2WithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.515750
DetX400,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### col_max_norm

In [18]:
separate_bits = False
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [19]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.7900,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.548000
Operation Crosstalk 1 (a),0.000,0.4795,0.78525,1.000000,0.91775,0.77925,0.869643,1.000000,0.808250
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,1.000000,0.99875,0.50075,0.795437,1.000000,0.516250
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.99900,0.999603,1.000000,0.999000
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.570000
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### row_norm

In [20]:
separate_bits = False
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [21]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.6940,0.51575,0.910000,0.65250,0.48800,0.795437,0.909623,0.504750
Operation Crosstalk 1 (a),0.000,0.4795,0.68925,0.917800,0.55375,0.68825,0.868552,0.918155,0.690750
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,0.910600,0.66025,0.50075,0.795437,0.910714,0.499500
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,0.91250,0.91250,0.790112,0.518346,0.911000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.65275,0.861111,0.909623,0.707750
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,0.909623,0.509750
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,0.779889,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,0.915079
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


## Expts with bit frequencies separated

### max_norm

In [31]:
separate_bits = True
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [32]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.99325,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.504750
Operation Crosstalk 1 (a),0.000,0.47950,0.99050,1.000000,1.00000,0.99075,0.992460,1.000000,0.988250
Operation Crosstalk 2 (b),0.000,0.00000,0.48550,1.000000,0.99850,0.50075,0.795437,1.000000,0.500250
Detection Crosstalk (c),0.000,0.00000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.00000,0.00000,0.000000,0.48800,0.99875,0.999702,1.000000,0.999000
XT2WithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.515750
DetX400,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### col_max_norm

In [33]:
separate_bits = True
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [34]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.7900,0.51575,1.000000,0.99875,0.48800,0.795437,1.000000,0.548000
Operation Crosstalk 1 (a),0.000,0.4795,0.78525,1.000000,0.91775,0.77925,0.869643,1.000000,0.808250
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,1.000000,0.99875,0.50075,0.795437,1.000000,0.516250
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,1.00000,1.00000,1.000000,0.518346,1.000000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.99900,0.999603,1.000000,0.999000
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,1.000000,0.570000
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,1.000000,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,1.000000
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


### row_norm

In [26]:
separate_bits = True
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [27]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

,IdealGates,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
IdealGates,0.488,0.6940,0.51575,0.910000,0.65250,0.48800,0.795437,0.909623,0.504750
Operation Crosstalk 1 (a),0.000,0.4795,0.68925,0.917800,0.55375,0.68825,0.868552,0.918155,0.690750
Operation Crosstalk 2 (b),0.000,0.0000,0.48550,0.910600,0.66025,0.50075,0.795437,0.910714,0.499500
Detection Crosstalk (c),0.000,0.0000,0.00000,0.487875,0.91250,0.91250,0.790112,0.518346,0.911000
XT1WithGates,0.000,0.0000,0.00000,0.000000,0.48800,0.65275,0.861111,0.909623,0.707750
XT2WithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.48800,0.795437,0.909623,0.509750
DetX400,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.492450,0.779889,0.804067
DetXWithGates,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.491275,0.915079
DetX100,0.000,0.0000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.487750


## Transfer Learning

In [13]:
def transfer_svm_expt(freq_1, freq_2, freq_3):
  # freq_1 gets label 1, freq_2 gets label 0
  # freq_3 is assumed to have label 1 (this is the dataset we are transferrig to)
  # No need for a test set since we are performing transfer learning
  X_train, X_test, y_train, y_test = binary_comparison_data_pipeline(freq_1, freq_2, test_frac = 0.0)

  clf = svm.SVC()
  clf.fit(X_train, y_train)

  X_transfer, y_transfer = create_dataset(freq_3, 1)

  transfer_acc = (clf.predict(X_transfer) == y_transfer).mean()

  return transfer_acc

In [42]:
# Apply binary expt to all possible combinations of data_list and pretty print

# Rows - trained on
# Columns - transferred to

def create_transfer_expt_matrix(dataset_list, data_list, binary_expt):
  num_datasets = len(data_list) 

  # get names of datasets
  dataset_names = [d['dir_name'] for d in dataset_list]

  expt_matrix = np.zeros((num_datasets - 1, num_datasets - 1))

  for i in range(1, num_datasets):
    for j in range(1, num_datasets):
      expt_matrix[i-1][j-1] = binary_expt(data_list[i][1], data_list[0][1], data_list[j][1])

  # pretty printing
  df = pd.DataFrame(expt_matrix, index=dataset_names[1:], columns=dataset_names[1:])
  display(df)

  return expt_matrix

In [43]:
separate_bits = True
norm = "row_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [44]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.4737,0.1069,0.111650,0.4236,0.1133,0.112203,0.112302,0.1110
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.4003,0.4437,1.000000,0.4616,0.4616,0.442252,1.000000,0.4632
XT1WithGates,0.3482,0.0000,0.000000,0.3097,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.310325,0.2847,0.2847,0.297005,0.306559,0.2924
DetX400,1.0000,1.0000,0.723325,1.0000,1.0000,1.000000,0.739876,1.0000
DetXWithGates,0.4003,0.4437,1.000000,0.4616,0.4616,0.442252,1.000000,0.4632
DetX100,0.6869,0.6957,0.661375,0.6936,0.6922,0.679604,0.685149,0.7051


In [45]:
separate_bits = True
norm = "max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [46]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.9798,0.0000,0.884550,0.0764,0.0000,0.000000,0.893391,0.0000
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
XT1WithGates,0.2871,0.0000,0.000000,0.9981,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.310325,0.2847,0.2847,0.297005,0.306559,0.2924
DetX400,1.0000,1.0000,0.946500,0.9851,1.0000,1.000000,0.948119,1.0000
DetXWithGates,0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
DetX100,0.6994,0.6957,0.692575,0.6921,0.6922,0.679604,0.715149,0.7051


In [47]:
separate_bits = True
norm = "col_max_norm"

data_list = setup_datalist(datasets, dir_prefix, file_prefix, separate_bits, norm)

In [48]:
svm_expt_matrix = create_transfer_expt_matrix(datasets, data_list, transfer_svm_expt)

,Operation Crosstalk 1 (a),Operation Crosstalk 2 (b),Detection Crosstalk (c),XT1WithGates,XT2WithGates,DetX400,DetXWithGates,DetX100
Operation Crosstalk 1 (a),0.6659,0.1188,0.111650,0.7027,0.1270,0.112228,0.112302,0.1111
Operation Crosstalk 2 (b),0.4256,0.3977,0.395625,0.3689,0.3689,0.386584,0.377277,0.3738
Detection Crosstalk (c),0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
XT1WithGates,0.4347,0.0000,0.000000,0.9984,0.0000,0.000000,0.000000,0.0000
XT2WithGates,0.3455,0.3036,0.188850,0.2847,0.2847,0.297005,0.189975,0.2924
DetX400,1.0000,1.0000,0.061950,1.0000,1.0000,1.000000,0.054728,1.0000
DetXWithGates,0.0000,0.0000,1.000000,0.0000,0.0000,0.000000,1.000000,0.0000
DetX100,0.4622,0.4952,0.031200,0.6282,0.5114,0.679431,0.030000,0.7051
